In [1]:
from gwpy.table import EventTable
from gwpy.time import to_gps
from io import StringIO, BytesIO
from matplotlib import pyplot as plt
from ligo.skymap.io import fits
from ligo.skymap import postprocess
import ligo.skymap.plot
import base64

In [2]:
t1 = "2017-08-17T12:40:54" 
t2 = "2017-08-25T12:41:10"

In [3]:
t1 = to_gps(t1)
t2 = to_gps(t2)

In [4]:
for catalog in ("GWTC-1-confident", "GWTC-2"): # , "O3_Discovery_Papers"):
    tab = EventTable.fetch_open_data(catalog, selection=f'GPS > {t1} && GPS < {t2}')
    odatab = tab['commonName', 
                 'GPS',
                 'luminosity_distance',
#                 'luminosity_distance_lower',
#                 'luminosity_distance_upper'
                 'mass_1_source', 
#                 'mass_1_source_lower', 
#                 'mass_1_source_upper', 
                 'mass_2_source', 
#                 'mass_2_source_lower', 
#                 'mass_2_source_upper', 
                ]
    if odatab:
        break

In [5]:
odatab.meta = {}

In [6]:
with StringIO() as fd:
    odatab.write(fd, format='ascii.ecsv')
    asciicat = fd.getvalue()

In [ ]:
plt.figure(figsize=(9, 6))
ax = plt.axes(projection=('astro hours mollweide'))
lpr = []
names = []
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
i = 0

for event in odatab:
    if event['commonName'] == 'GW170817':
        ax.plot(197.45035417, -23.38148417,
                transform=ax.get_transform('world'),
                marker='+',
                markersize=10,
                markeredgewidth=2, 
                color=colors[i])    
        lpr.append(plt.Rectangle((0,0),1,1,fc = colors[i]))
        names.append('GW170817')
    else:
        fits_file = f"./skymaps/{event['commonName']}.fits"
        contour_levels = [50,90]
        try:
            skymap, metadata = fits.read_sky_map(fits_file, nest=None)
            cls = 100 * postprocess.find_greedy_credible_levels(skymap)
            cs = ax.contour_hpx(
                        (cls, 'ICRS'), nested=metadata['nest'],
                         colors=colors[i], linewidths=1, levels=contour_levels)
            lpr.append(plt.Rectangle((0,0),1,1,fc = colors[i]))
            names.append(event['commonName'])
        except:
            pass #this could be done better. At least find the way to notify user of missed contours.
    i+=1

ax.legend(lpr, names)
plt.grid(True)

bio = BytesIO()
plt.savefig(bio, format='jpg')
bio.seek(0)
jpgdata = base64.b64encode(bio.read())

In [13]:
asciicat = asciicat
jpgdata = jpgdata.decode()